In [1]:
import tensorflow as tf
import numpy as np
from ptr_decoder import pointer_decoder
from pointer_network import PointerNetwork
from data_generator import *
%load_ext autoreload
%autoreload 2

In [2]:
input_dim = 1
batch_size = 2
seq_len = 8
hidden_dim = 100
lr = 0.001
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
pointer_net = PointerNetwork(hidden_dim, lr, 5, seq_len, batch_size=batch_size, input_dim=1)

In [4]:
opt = pointer_net.train_op
loss = pointer_net.loss
pred = pointer_net.predictions
decoder_outputs = pointer_net.decoder_outputs
sess.run(tf.initialize_all_variables())

In [ ]:
for i in range(0, 10000):
    _encoder_inps, _targets, _decoder_inps = sorting_generator(seq_len, batch_size)
    # _encoder_inps, _decoder_inps, _targets = data.next_batch(2, 8)
    feed_dict = pointer_net.feed_dict(decoder_inpt_data=_decoder_inps, 
                                      encoder_inpt_data=_encoder_inps, target_data=_targets)

    _loss, _, _vs = sess.run([loss, opt, pred], feed_dict=feed_dict)
    if i%100 == 0:    
        print np.argmax(np.array(_targets)[:, 0, :], axis=1)
        print np.argmax(np.array(_vs)[:, 0, :], axis=1)
        print "Loss", _loss

[4 1 0 7 3 4 7 2 8]
[8 8 8 8 8 8 8 8 8]
Loss 25.6083
[1 6 6 3 4 5 0 7 8]
[1 8 8 8 8 8 8 8 8]
Loss 3.10769
[1 0 6 6 0 2 5 5 8]
[0 0 0 7 8 8 8 8 8]
Loss 3.07048
[3 3 7 6 2 2 6 5 8]
[0 0 0 8 8 8 8 8 8]
Loss 2.96422
[2 5 0 6 6 7 3 3 8]
[6 5 4 0 8 8 8 8 8]
Loss 2.91899
[5 4 3 7 1 5 0 3 8]
[5 5 5 1 8 8 8 8 8]
Loss 2.81097
[3 0 4 2 0 7 2 6 8]
[6 6 6 6 8 8 8 8 8]
Loss 2.70599
[5 3 0 1 3 2 6 5 8]
[5 5 5 6 8 8 8 8 8]
Loss 2.75207
[7 7 2 6 3 1 6 3 8]
[6 6 6 4 8 8 8 8 8]
Loss 2.68255
[4 7 1 1 5 4 7 6 8]
[6 6 6 2 8 8 8 8 8]
Loss 2.73916
[4 6 6 7 3 1 2 0 8]
[6 6 6 6 8 8 8 8 8]
Loss 2.73892
[5 5 4 6 3 7 0 4 8]
[7 3 2 0 8 8 8 8 8]
Loss 2.68974
[2 5 7 2 5 7 4 3 8]
[7 2 7 7 8 8 8 8 8]
Loss 2.64207
[2 1 3 5 0 2 5 7 8]
[0 0 0 4 8 8 8 8 8]
Loss 2.70168
[5 4 4 5 7 2 2 3 8]
[7 2 2 7 8 8 8 8 8]
Loss 2.72995
[5 0 0 3 2 5 6 2 8]
[5 0 0 4 8 8 8 8 8]
Loss 2.67354
[0 2 7 6 3 3 1 4 8]
[7 7 7 7 8 8 8 8 8]
Loss 2.72896
[1 5 0 4 5 3 6 7 8]
[6 6 7 2 8 8 8 8 8]
Loss 2.67322
[0 3 5 2 7 5 3 6 8]
[0 0 0 2 8 8 8 8 8]
Loss 2

In [6]:
import numpy as np


class DataGenerator(object):

    def __init__(self):
        """Construct a DataGenerator."""
        pass

    def next_batch(self, batch_size, N, train_mode=True):
        """Return the next `batch_size` examples from this data set."""

        # A sequence of random numbers from [0, 1]
        reader_input_batch = []

        # Sorted sequence that we feed to encoder
        # In inference we feed an unordered sequence again
        decoder_input_batch = []

        # Ordered sequence where one hot vector encodes position in the input array
        writer_outputs_batch = []
        for _ in range(N):
            reader_input_batch.append(np.zeros([batch_size, 1]))
        for _ in range(N + 1):
            decoder_input_batch.append(np.zeros([batch_size, 1]))
            writer_outputs_batch.append(np.zeros([batch_size, N + 1]))

        for b in range(batch_size):
            shuffle = np.random.permutation(N)
            sequence = np.sort(np.random.random(N))
            shuffled_sequence = sequence[shuffle]

            for i in range(N):
                reader_input_batch[i][b] = shuffled_sequence[i]
                if train_mode:
                    decoder_input_batch[i + 1][b] = sequence[i]
                else:
                    decoder_input_batch[i + 1][b] = shuffled_sequence[i]
                writer_outputs_batch[shuffle[i]][b, i + 1] = 1.0

            # Points to the stop symbol
            writer_outputs_batch[N][b, -1] = 1.0

        return reader_input_batch, decoder_input_batch, writer_outputs_batch
